In [1]:
!pip install numpy scipy scikit-learn mne 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 36.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 9.3 MB/s eta 0:00:00


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.signal import butter, lfilter
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
train_cnt_file = "train/Competition_train_cnt.txt"
train_lab_file = "train/Competition_train_lab.txt"

train_cnt = np.loadtxt(train_cnt_file)
Y_train = np.loadtxt(train_lab_file)

print("Raw shape of train_cnt:", train_cnt.shape)


Raw shape of train_cnt: (17792, 3000)


In [3]:
n_trials_train = 278
n_channels = 64
n_samples = 3000

X_train = train_cnt.reshape(n_trials_train, n_channels, n_samples)

print("X_train final shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)

X_train final shape: (278, 64, 3000)
Y_train shape: (278,)


In [4]:
test_file = "test.txt"
test_label_file = "test_label.txt"

test_cnt = np.loadtxt(test_file)
Y_test = np.loadtxt(test_label_file)

print("Raw shape of test_cnt:", test_cnt.shape)

Raw shape of test_cnt: (6400, 3000)


In [5]:
n_trials_test = 100
X_test = test_cnt.reshape(n_trials_test, n_channels, n_samples)

print("test final shape:", X_test.shape)

test final shape: (100, 64, 3000)


In [6]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

print("Train Set Shape:", X_train.shape, Y_train.shape)
print("Validation Set Shape:", X_val.shape, Y_val.shape)

Train Set Shape: (222, 64, 3000) (222,)
Validation Set Shape: (56, 64, 3000) (56,)


In [7]:
def bandpass_filter(data, lowcut=8, highcut=30, fs=1000, order=5):
    """
    Apply a bandpass filter to the input data.

    Parameters:
        data (ndarray): EEG/ECoG data of shape (trials, channels, samples)
        lowcut (float): Lower cutoff frequency (default: 8 Hz)
        highcut (float): Upper cutoff frequency (default: 30 Hz)
        fs (int): Sampling frequency (default: 1000 Hz)
        order (int): Order of the filter (default: 5)

    Returns:
        ndarray: Filtered data (same shape as input)
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')

    filtered_data = lfilter(b, a, data, axis=-1)
    return filtered_data


In [8]:
X_train_filt = bandpass_filter(X_train)
X_val_filt = bandpass_filter(X_val)
X_test_filt = bandpass_filter(X_test)

In [9]:
def compute_covariance(trial_data):
    """
    Compute the covariance matrix for a given trial.
    
    Parameters:
        trial_data (ndarray): Shape (channels, samples)
    
    Returns:
        ndarray: Covariance matrix (channels x channels)
    """
    return np.cov(trial_data)

def csp_fit(X, y, n_components=3):
    """
    Fit CSP spatial filters.

    Parameters:
        X (ndarray): Shape (trials, channels, samples)
        y (ndarray): Labels (-1 or 1), shape (trials,)
        n_components (int): Number of CSP components to retain per class

    Returns:
        ndarray: CSP spatial filters (channels, 2 * n_components)
    """

    X_class1 = X[y == -1]
    X_class2 = X[y == 1]

    cov_class1 = np.mean([compute_covariance(x) for x in X_class1], axis=0)
    cov_class2 = np.mean([compute_covariance(x) for x in X_class2], axis=0)

    from scipy.linalg import eigh
    w, v = eigh(cov_class1, cov_class2)

    idx = np.argsort(w)[::-1]
    v = v[:, idx]
    filters = np.hstack([v[:, :n_components], v[:, -n_components:]])
    
    return filters

def csp_transform(X, filters):
    """
    Apply CSP transformation to extract log-variance features.

    Parameters:
        X (ndarray): Shape (trials, channels, samples)
        filters (ndarray): CSP spatial filters (channels, 2*n_components)

    Returns:
        ndarray: CSP features (trials, 2*n_components)
    """
    n_trials, _, _ = X.shape
    n_filters = filters.shape[1]
    features = np.zeros((n_trials, n_filters))

    for i in range(n_trials):
        projected = filters.T @ X[i] 
        var = np.var(projected, axis=1)
        features[i, :] = np.log(var)

    return features


In [10]:
csp_filters = csp_fit(X_train_filt, Y_train, n_components=3)

X_train_csp = csp_transform(X_train_filt, csp_filters)
X_val_csp = csp_transform(X_val_filt, csp_filters)
X_test_csp = csp_transform(X_test_filt, csp_filters)

clf = LinearDiscriminantAnalysis()
clf.fit(X_train_csp, Y_train)

pred_val = clf.predict(X_val_csp)
acc_val = accuracy_score(Y_val, pred_val)
print(f"Validation Accuracy: {acc_val:.4f}")


Validation Accuracy: 0.8929


In [11]:
pred_noadapt = clf.predict(X_test_csp)

acc_noadapt = accuracy_score(Y_test, pred_noadapt)
print(f"No Adaptation Test Accuracy: {acc_noadapt:.4f}")

print("Confusion Matrix (No Adaptation):")
print(confusion_matrix(Y_test, pred_noadapt))


No Adaptation Test Accuracy: 0.7600
Confusion Matrix (No Adaptation):
[[50  0]
 [24 26]]
